<a href="https://colab.research.google.com/github/ljmcpherson/ISP_BCI/blob/main/RawDataAnalysis_31Oct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
import scipy.io
import numpy as np

In [ ]:
mat_file = scipy.io.loadmat('/content/drive/Shareddrives/Impact Scholars /Computational Neuroscience/Akito Adaptive Modeling/Raw Data/motor_basic/motor_basic/data/bp_mot_t_h.mat')
#mat_file = scipy.io.loadmat('sub2_comp.mat')


In [ ]:
# Print the keys in the .mat file
print(mat_file.keys())

dict_keys(['__header__', '__version__', '__globals__', 'data', 'stim'])


In [ ]:
# Extract the variable of interest
ecog_signal = mat_file['data']
stim = mat_file['stim']

In [ ]:
count = np.sum(stim == 12)
print(count)

90000


In [ ]:
def find_consecutive_ones_with_indices(arr, temp):
    max_count = 0
    current_count = 0
    max_start_index = -1
    max_end_index = -1
    current_start_index = 0

    for i, num in enumerate(arr):
        if num == temp:
            if current_count == 0:
                current_start_index = i
            current_count += 1
            if current_count > max_count:
                max_count = current_count
                max_start_index = current_start_index
                max_end_index = i
        else:
            current_count = 0

    return max_count, max_start_index, max_end_index


In [ ]:
# Find the maximum length of consecutive 1s and their indices
max_consecutive_ones, start_index, end_index = find_consecutive_ones_with_indices(stim, 11)
print(f'Maximum length of consecutive 11s: {max_consecutive_ones}')
print(f'Start index: {start_index}')
print(f'End index: {end_index}')
# Find the maximum length of consecutive 1s and their indices
max_consecutive_ones, start_index, end_index = find_consecutive_ones_with_indices(stim, 12)
print(f'Maximum length of consecutive 12s: {max_consecutive_ones}')
print(f'Start index: {start_index}')
print(f'End index: {end_index}')

Maximum length of consecutive 11s: 3000
Start index: 28240
End index: 31239
Maximum length of consecutive 12s: 3000
Start index: 10160
End index: 13159


In [ ]:
from scipy.signal import butter, filtfilt, iirnotch

In [ ]:
# Bandpass filter (e.g., 1-150 Hz)
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Apply the bandpass filter
filtered_signal = bandpass_filter(ecog_signal, 1, 150, 1000)


In [ ]:
# Design a notch filter
f0 = 50.0  # Frequency to be removed from signal (Hz)
Q = 30.0   # Quality factor
fs = 1000.0  # Sampling frequency (Hz)

# Create the notch filter
b, a = iirnotch(f0, Q, fs)

# Apply the notch filter
notch_signal = filtfilt(b, a, filtered_signal)



In [ ]:
!pip install mne
import mne

In [ ]:
# Alternatively, truncating to the shortest length
min_length = min(len(channel) for channel in ecog_signal)
truncated_signal = np.array([channel[:min_length] for channel in ecog_signal])
print('Shape of truncated_signal:', truncated_signal.shape)

Shape of truncated_signal: (376400, 47)


In [ ]:
def PSD():
  from scipy.signal import welch, find_peaks

  sampling_rate = 1000  # Hz
  data = ecog_signal
  n_channels = data.shape[1]
  # Perform spectral analysis on each channel and average the results
  frequencies, power = welch(data[:, 0], fs=sampling_rate, nperseg=1024)
  avg_power = np.zeros_like(power)

  for i in range(n_channels):
    _, power = welch(data[:, i], fs=sampling_rate, nperseg=1024)
    avg_power += power

  avg_power /= n_channels

  # Find peaks in the averaged power spectrum
  peaks, _ = find_peaks(avg_power, height=0.1)  # Adjust height threshold as needed
  return power, frequencies.flatten()
# Plot the power spectrum and mark the peaks
#plt.semilogy(frequencies, avg_power)
#plt.plot(frequencies[peaks], avg_power[peaks], 'ro')
#plt.xlabel('Frequency (Hz)')
#plt.ylabel('Power')
#plt.title('Spectral Analysis of ECoG Data')
#plt.show()

# Print the identified frequency ranges
#print("Identified frequency ranges (Hz):", frequencies[peaks])


In [ ]:
#@title SSD Implementation
from scipy.linalg import eigh

class SSD:
    def __init__(self, n_components=25):
        self.n_components = n_components
        self.filters_ = None

    def fit(self, X, sampling_rate, bandpass_freq):
        # Band-pass filter the data
        X_bandpass = self._bandpass_filter(X, sampling_rate, bandpass_freq)

        # Compute covariance matrices
        C_bandpass = np.cov(X_bandpass.T)
        C_raw = np.cov(X.T)

        # Solve the generalized eigenvalue problem
        _, V = eigh(C_bandpass, C_raw)

        # Select the top n_components eigenvectors
        self.filters_ = V[:, :self.n_components]

    def transform(self, X):
        # Apply spatial filters to the data
        return X @ self.filters_

    def _bandpass_filter(self, X, sampling_rate, bandpass_freq):
        from scipy.signal import butter, lfilter

        nyquist = 0.5 * sampling_rate
        low, high = bandpass_freq[0] / nyquist, bandpass_freq[1] / nyquist
        b, a = butter(1, [low, high], btype='band')

        return lfilter(b, a, X, axis=0)

# Example usage
# X is a 2D array with shape (n_timepoints, n_channels)
# sampling_rate is the sampling rate of the data
# bandpass_freq is a tuple of low and high frequencies for band-pass filtering
X = ecog_signal # Example ECoG data
sampling_rate = 1000  # Example sampling rate in Hz
bandpass_freq = (0.1, 100)  # Example band-pass filter range in Hz

ssd = SSD(n_components=25)
ssd.fit(X, sampling_rate, bandpass_freq)
X_transformed = ssd.transform(X)

print(X_transformed.shape)  # Output shape: (300000, 2)


In [ ]:
import matplotlib.pyplot as plt

# Assuming X_transformed is the SSD-transformed data with shape (n_timepoints, n_components)
  # Replace with your actual transformed data

for i in range(X_transformed.shape[1]):
# Plot the first component
  plt.figure(figsize=(15, 5))
  plt.plot(X_transformed[:, i], label=f'Component {i+1}')
  plt.xlabel('Time Points')
  plt.ylabel('Amplitude')
  plt.title('SSD Component 1')
  plt.legend()
  plt.show()



In [ ]:
def compute_snr(components):
  snr = []
  for component in components.T:
    signal_power = np.mean(component**2)
    noise_power = np.var(component - np.mean(component))
    snr.append(10 * np.log10(signal_power / noise_power))
  return snr

In [ ]:
snr = compute_snr(X_transformed)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(snr) + 1), snr, marker='o')
plt.xlabel('Component')
plt.ylabel('SNR (dB)')
plt.title('SNR of SSD Components')
plt.grid(True)
plt.show()

In [ ]:
import sys
sys.path.insert(0,'/content/drive/Shareddrives/Impact Scholars /Computational Neuroscience/Akito Adaptive Modeling/Program Components')

In [ ]:
import ssd

In [ ]:
!pip install pyvista

In [ ]:
!pip install fooof

In [ ]:
import fooof

In [ ]:
import helper

In [ ]:
from ssd import run_ssd, apply_filters

In [ ]:
ecog_signal.shape

(376400, 47)

In [ ]:
from mne.time_frequency import psd_array_welch

In [ ]:
def get_SNR(raw, fmin=1, fmax=55, seconds=3, freq=[8, 13]):
    """Compute power spectrum and calculate 1/f-corrected SNR in one band.

    Parameters
    ----------
    raw : instance of Raw
        Raw instance containing traces for which to compute SNR
    fmin : float
        minimum frequency that is used for fitting spectral model.
    fmax : float
        maximum frequency that is used for fitting spectral model.
    seconds: float
        Window length in seconds, converts to FFT points for PSD calculation.
    freq : list | [8, 13]
        SNR in that frequency window is computed.
    Returns
    -------
    SNR : array, 1-D
        Contains SNR (1/f-corrected, for a chosen frequency) for each channel.
    """
    SNR = np.zeros(((376400, 47)))
    n_fft = int(seconds * raw.info["sfreq"])
    psd, freqs = PSD()
    print(freqs.shape)
    print(psd.shape)
    fm = fooof.FOOOFGroup()
    fm.fit(freqs, psd)

    for pick in range(len(raw.ch_names)):
        psd_corr = 10 * np.log10(psd[pick]) - 10 * fm.get_fooof(pick)._ap_fit
        idx = np.where((freqs > freq[0]) & (freqs < freq[1]))[0]
        idx_max = np.argmax(psd_corr[idx])
        SNR[pick] = psd_corr[idx][idx_max]

    return SNR


In [ ]:
#@title Natalie paper
import numpy as np
import helper

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


plt.ion()
exp_id = "motor_basic"
participant = "ug"

# -- load continuous data
# Create MNE Info object
ch_names = [f'Ch{i}' for i in range(1, ecog_signal.shape[1] + 1)]  # Example channel names
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types='ecog')
data = ecog_signal
# Create Raw object
raw = mne.io.RawArray(data.T, info)
raw.pick_types(ecog=True)
raw_org = raw.copy()

# -- apply SSD and compute SNR for all components
bin_width = 1.2
peak = 9.46
nr_components = 25

filters, patterns = ssd.run_ssd(raw_org, peak, bin_width)
raw_ssd = ssd.apply_filters(raw_org, filters)

# -- SSD is polarity invariant, align to electrode signals by visual inspection
signs = [1, -1, -1]
for i_s, sign in enumerate(signs):
    patterns[:, i_s] *= sign
    raw_ssd._data[i_s] *= sign


SNR_ssd = get_SNR(raw_ssd, freq=[peak - 2, peak + 2])
raw_ssd.pick(range(nr_components))

# -- select electrodes and compute SNR for the selected peak frequency
ch_names = ["ecog16", "ecog10", "ecog20"]
picks = mne.pick_channels(raw.ch_names, ch_names, ordered=True)
raw.pick(picks)
SNR_electrodes = helper.get_SNR(raw, freq=[peak - 2, peak + 2])

# -- create figure
fig = plt.figure()
outer_grid = gridspec.GridSpec(2, 1, height_ratios=[2.5, 1])
top_cell = outer_grid[0, :]
bottom_cell = outer_grid[1, :]
gs = gridspec.GridSpecFromSubplotSpec(
    2, 3, top_cell, width_ratios=[1.25, 2.5, 1]
)

# -- plot electrode PSD + SNR
ax1 = plt.subplot(gs[0, 0])
cmap1 = ["#2d004f", "#254f00", "#000000"]
helper.plot_psd(
    ax1,
    raw,
    cmap=cmap1,
    SNR=SNR_electrodes,
    peak=peak,
    bin_width=bin_width,
)

# -- plot PSD for SSD component
ax1 = plt.subplot(gs[1, 0])
cmap = [plt.cm.viridis(i) for i in np.linspace(0.2, 1, 4)]

helper.plot_psd(
    ax1, raw_ssd, cmap=cmap, SNR=SNR_ssd, peak=peak, bin_width=bin_width
)

# -- plot time domain signals
tmin = 4
tmax = tmin + 2

raw_ssd.filter(2, None)
raw.filter(2, None)

raw.crop(tmin, tmax)
raw_ssd.crop(tmin, tmax)

# -- plot electrode signals
ax1 = plt.subplot(gs[0, 1])
helper.plot_timeseries(ax1, raw, cmap=cmap1, label="")
ax1.set_xlabel("time [ms]")


# -- pattern coefficients
ax_cont = plt.subplot(gs[0, 2])

contributions = patterns[picks, :nr_components]
vmax = np.max(np.abs(contributions))
ax_cont.imshow(contributions, cmap="RdBu_r", vmax=vmax, vmin=-vmax)
ax_cont.set(yticks=[])
ax_cont.set_xticks(range(nr_components))
ax_cont.set_xticklabels(["comp.%i" % (i + 1) for i in range(nr_components)])


for (j, i), label in np.ndenumerate(contributions):
    ax_cont.text(i, j, "%.2f" % label, ha="center", va="center")


# -- plot SSD time series
ax1 = plt.subplot(gs[1, 1])
helper.plot_timeseries(ax1, raw_ssd, cmap=cmap, label="")
ax1.set_xlabel("time [ms]")
for i_chan in range(nr_components):
    ax1.text(0, -i_chan + 0.35, "component %i" % (i_chan + 1))


# -- plot SNR for SSD components
ax1 = plt.subplot(gs[1, 2])
ax1.plot(SNR_ssd, ".-", color="k", markeredgecolor="w", markersize=8)
for i in range(nr_components):
    ax1.plot(
        i, SNR_ssd[i], ".", color=cmap[i], markeredgecolor="w", markersize=12
    )
ax1.set(xlabel="component number", ylabel="SNR [dB]")


# -- plot filters
gs2 = gridspec.GridSpecFromSubplotSpec(1, 7, bottom_cell)
topo_size = 0.26

for i in range(nr_components):

    ax = plt.Axes(fig, rect=[i * 0.18 + 0.35, 0.12, topo_size, topo_size])
    ax1 = fig.add_axes(ax)

    im_filters = helper.make_topoplot(
        filters[:, i],
        raw_org.info,
        ax,
        plot_head=False,
        picks=picks,
        cmap="PiYG",
        pick_color=["dimgrey"],
        vmin=-0.75,
        vmax=0.75,
    )

    ax1.set_ylim(-0.04, 0.04)
    ax1.set_xlim(-0.08, 0.08)

# -- plot patterns
for i in range(nr_components):

    ax = plt.Axes(fig, rect=[i * 0.18 + 0.35, -0.05, topo_size, topo_size])
    ax1 = fig.add_axes(ax)

    im_patterns = helper.make_topoplot(
        patterns[:, i],
        raw_org.info,
        ax1,
        plot_head=False,
        picks=picks,
        cmap="RdBu_r",
        vmin=-1.5,
        vmax=1.5,
        pick_color=["dimgrey"],
    )

    ax1.set_title("component %i" % (i + 1))
    ax1.set_ylim(-0.04, 0.04)
    ax1.set_xlim(-0.08, 0.08)


# -- filters & patterns colorbars
ax = plt.Axes(fig, rect=[0.3, 0.225, 0.15, 0.025])
ax1 = fig.add_axes(ax)
cb = plt.colorbar(im_filters, cax=ax1, orientation="horizontal")
ax1.set_title("spatial filters")

ax = plt.Axes(fig, rect=[0.3, 0.05, 0.15, 0.025])
ax1 = fig.add_axes(ax)
cb = plt.colorbar(im_patterns, cax=ax1, orientation="horizontal")
ax1.set_title("spatial patterns")

# -- electrodes on topo head
ax = plt.Axes(fig, rect=[0.0, 0.02, topo_size, topo_size])
ax1 = fig.add_axes(ax)
mask = np.zeros((len(raw_org.ch_names)), dtype="bool")
mask[picks] = True
mne.viz.plot_topomap(
    np.zeros((len(raw_org.ch_names),)) + np.nan,
    raw_org.info,
    axes=ax1,
    mask=mask,
)

fig.set_size_inches(7.5, 6)

fig.show()

Creating RawArray with float64 data, n_channels=47, n_times=376400
    Range : 0 ... 376399 =      0.000 ...   376.399 secs
Ready.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
(513,)
(513,)


DataError: Inputs are not the right dimensions.